In [1]:
from nlpinitiative.data_preparation import data_import, data_preparation, dataset_normalizer
from nlpinitiative.config import (
    EXTERNAL_DATA_DIR, 
    CONV_SCHEMA_DIR
)

2025-02-12 11:19:13.925 | INFO     | nlpinitiative.config:<module>:11 - PROJ_ROOT path is: C:\Users\Daniel\Desktop\GitHub\NLPinitiative
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Importing third-party datasets for use in NLP model training:

For the purposes of this project, we are going to rely on third-party datasets to make up for a lack of personally procured data. As such, we have implemented some functionality to make this easier for future developers/data analysts.

### Importing datasets from a local source (on your local system):
For the purposes of our applications, we will consider the "raw" datasets to be personally produced datasets rather than those that have already been created ("external"). As such, importing from a local source will by default store the datasets within the data/raw directory. If the data to be imported locally is a third-party dataset, the user can change the 'tp_src' value to True, where the data will be stored within the data/external directory.

In [ ]:
local_import_example_df = data_import.import_from_local_source("C:/Users/Daniel/Downloads/dataset.csv", tp_src=False)
print(local_import_example_df)

### Demonstration of importing from remote/external source:
This function facillitates importing data from a given URL (primarily remote repositories like GitHub).

In [ ]:
remote_import_example_df = data_import.import_from_ext_source("https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset/blob/master/ethos/ethos_data/Ethos_Dataset_Binary.csv")
print(remote_import_example_df)

# Conversion of third-party datasets:
Since we are making use of third-party datasets, we need a means of converting the original dataset schema to a format that will utilize our labeling scheme. As such, we have implemented some functionality to facilitate this process.

### Normalizing third-party datasets to a standard format for our applications:
This function facilitates taking one or more datasets (all passed datasets that are to be normalized should maintain the same general structure so that they can be merged prior to normalization).

In [ ]:
srcs = [
    EXTERNAL_DATA_DIR / "intelligence-csd-auth-gr_Ethos-Hate-Speech-Dataset_Ethos_Dataset_Binary.csv",
    EXTERNAL_DATA_DIR / "intelligence-csd-auth-gr_Ethos-Hate-Speech-Dataset_Ethos_Dataset_Multi_Label.csv"
]
conv = CONV_SCHEMA_DIR / "ethos_schema_mapping.json"

normalized_dataset = dataset_normalizer.convert_to_master_schema(srcs, conv, 'ETHOS_dataset_converted')
print(normalized_dataset)

# Data Preparation:
Prior to being able to use the dataset in training the model, we must first perpare the data by converting it into a dataset, and tokenizing the textual data (in addition to restructuring the data to a format that can be passed into a model).

### Loading csv as Dataset object:

In [2]:
dataset = data_preparation.get_dataset_from_file("ETHOS_dataset_converted.csv")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'DISCRIMINATORY', 'NEUTRAL', 'GENDER', 'RACE', 'SEXUALITY', 'DISABILITY', 'RELIGION', 'UNSPECIFIED'],
        num_rows: 698
    })
    test: Dataset({
        features: ['TEXT', 'DISCRIMINATORY', 'NEUTRAL', 'GENDER', 'RACE', 'SEXUALITY', 'DISABILITY', 'RELIGION', 'UNSPECIFIED'],
        num_rows: 300
    })
})


### Extracting labels and initializing dicts for converting from labels to ids and ids to labels:

In [3]:
labels, lbl2idx, idx2lbl = data_preparation.get_labels_and_dicts(dataset)
print(labels)
print(lbl2idx)
print(idx2lbl)

['DISCRIMINATORY', 'NEUTRAL', 'GENDER', 'RACE', 'SEXUALITY', 'DISABILITY', 'RELIGION', 'UNSPECIFIED']
{'DISCRIMINATORY': 0, 'NEUTRAL': 1, 'GENDER': 2, 'RACE': 3, 'SEXUALITY': 4, 'DISABILITY': 5, 'RELIGION': 6, 'UNSPECIFIED': 7}
{0: 'DISCRIMINATORY', 1: 'NEUTRAL', 2: 'GENDER', 3: 'RACE', 4: 'SEXUALITY', 5: 'DISABILITY', 6: 'RELIGION', 7: 'UNSPECIFIED'}


### Initialization of a tokenizer (using pre-trained BERT tokenizer)

In [4]:
tokenizer = data_preparation.get_tokenizer()
encoded_text = tokenizer.encode("Testing tokenizer for encoding")
decoded_text = tokenizer.decode(encoded_text)
print(encoded_text)
print(decoded_text)

[101, 5604, 19204, 17629, 2005, 17181, 102]
[CLS] testing tokenizer for encoding [SEP]


### Encode (preprocess) the dataset:

In [ ]:
ecd_dataset = data_preparation.preprocess_dataset(dataset, labels, tokenizer)
dataset_entry_ex = ecd_dataset['train'][0]
print(dataset_entry_ex.keys())

{'input_ids': [[101, 2012, 2008, 2391, 2017, 2031, 1037, 2157, 2000, 6985, 4426, 1012, 1012, 1012, 2021, 1996, 6517, 2518, 2003, 1045, 6655, 2065, 2002, 2052, 2031, 8047, 9599, 2002, 2052, 2031, 5407, 1999, 4390, 1012, 1012, 1012, 13718, 2002, 2106, 1996, 2157, 2518, 2021, 9015, 2094, 6551, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2123, 1005, 1056, 2113, 2021, 1045, 2123, 1005, 1056, 3404, 2023, 3124, 1005, 1055, 1012, 1012, 1012, 2009, 1005, 1055, 2035, 2055, 1996, 2769, 1012, 1012, 1012, 20228, 4757, 2292, 1996, 2757, 1999, 2037, 2088, 2644, 2437, 3065, 1012, 1012, 1012, 2007, 2010, 2331, 1012, 1012, 2522, 8557, 1057, 2134, 1005, 1056, 2031, 7395, 4757, 2015, 2043, 2002, 2001, 4142, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 